©2020-2021 ETH Zurich, Pagan Nicolò; D-ITET; Automatic Control Lab

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
import matplotlib.pyplot as plt
import os
from pytictoc import TicToc
import tikzplotlib
import scipy.special
import math

from functions import *
from simulation import Simulation, Simulation_parameters
from model import Model, Model_parameters

# Out-degree fitting analysis
In what follows, we provide the fitting analysis of the numerical out-degree distribution function. We use one simulation with 10^5 nodes. We first fit the data with a powerlaw and a lognormal distribution (the statistics for the fitting are printed).

In [ ]:
np.random.seed(0)
tt=TicToc()
tt.tic()
N = [100000]
nsim = 1
save_results = True
model_verbose = False
model_folder = 'numerical_outdegree_analysis_fitting/'
model_parameters = Model_parameters(N, nsim, save_results, model_folder, model_verbose)

T = np.NaN
pPA = 0.0
speed_up = True
simulation_verbose = False
simulation_parameters = Simulation_parameters(T, pPA, speed_up, simulation_verbose)

model = Model(model_parameters, simulation_parameters)
model.run()
outdegrees, numerical_dist, fit_alpha, fit_loc, fit_beta, powerlawfit, lognormalfit, mu, sigma = model.fitting_outdegree_analysis()
tt.toc()

Then, we plot the numerical probability density function and the powerlaw and lognormal fits. We also fit it with a gamma distribution, and with a Poisson distribution. Results are shown in Supplementary Fig. 2 (left).

In [ ]:
plt.rcParams["figure.figsize"]=[10,7]
n_tol = 50
# Numerical distribution plot
plt.scatter(range(min(N[0],n_tol)), numerical_dist.pdf(range(min(N[0],n_tol))), s=50, color='r', marker='o', label='Numerical data', alpha=0.5)

# Powerlaw distribution fit
powerlaw_fit_distribution = powerlaw.Power_Law(xmin=13, parameters=[powerlawfit[0].power_law.alpha], discrete=True) 
plt.loglog(range(int(powerlawfit[0].power_law.xmin),n_tol), powerlaw_fit_distribution.pdf(range(int(powerlawfit[0].power_law.xmin),n_tol)), color='b', label='fit pdf: alpha=' +str(powerlawfit[0].power_law.alpha)+' sigma='+str(powerlawfit[0].power_law.sigma)+' xmin='+str(int(powerlawfit[0].power_law.xmin)), alpha=0.5)

# Lognormal distribution fit
plt.plot(range(n_tol), lognormalfit[0].pdf(range(n_tol)), color='black', label='lognormal fit, mu='+str(mu[0])+' sigma='+str(sigma[0]), alpha=0.5)

# Gamma distribution fit
x = np.linspace(1,min(N[0],n_tol),min(N[0],n_tol))
y = stats.gamma.pdf(x, fit_alpha, fit_loc, fit_beta)
plt.plot(x, y, label='Fit with Gamma distribution, alpha='+str(fit_alpha)+'loc='+str(fit_loc)+'beta'+str(fit_beta))

# Poisson distribution fit
t = np.arange(0, min(N[0],n_tol), 1)
expected_outdegree = compute_expected_outdegree(N[0])
d = np.exp(-expected_outdegree[N[0]])*np.power(expected_outdegree[N[0]], t)/scipy.special.factorial(t)
plt.plot( t, d, 'bs', label='Poisson')

plt.xscale('log')
plt.yscale('log')
plt.xlim([1, n_tol])
plt.ylim([10**(-10),0.3])

plt.legend(loc="lower left")
if (save_results):
     tikzplotlib.save(model_folder+'outdegree_distribution_fit.tikz',  table_row_sep='\\\\\n')
plt.show()

For the same distributions (and fits) as above, we plot the complementary cumulative distribution functions. Results are reported in Supplementary Fig. 2 (right).

In [ ]:
plt.rcParams["figure.figsize"]=[10,7]
n_tol = 50

# Numerical distribution plot
plt.scatter(range(min(N[0],n_tol)), 1-numerical_dist.cdf(range(min(N[0],n_tol))), s=25, color='r', marker='o', label='Numerical data', alpha=0.5)

# Powerlaw fit
powerlaw_fit_distribution = powerlaw.Power_Law(xmin=13, parameters=[powerlawfit[0].power_law.alpha], discrete=True) 
plt.plot(range(int(powerlawfit[0].power_law.xmin),n_tol), 1-powerlaw_fit_distribution.cdf(range(int(powerlawfit[0].power_law.xmin),n_tol)), color='b', label='fit pdf: alpha=' +str(powerlawfit[0].power_law.alpha)+' sigma='+str(powerlawfit[0].power_law.sigma)+' xmin='+str(int(powerlawfit[0].power_law.xmin)), alpha=0.5)

# Lognormal fit
plt.plot(range(n_tol), 1-lognormalfit[0].cdf(range(n_tol)), color='black', label='lognormal fit, mu='+str(mu[0])+' sigma='+str(sigma[0]), alpha=0.5)

# Gamma distribution fit
x = np.linspace(0,min(N[0],n_tol),min(N[0],n_tol)+1)
y = stats.gamma.cdf(x, fit_alpha, fit_loc, fit_beta)
plt.plot(x, 1-y, label='Fit with Gamma distribution, alpha='+str(fit_alpha)+'loc='+str(fit_loc)+', beta='+str(fit_beta))

# Poisson distribution fit
plt.plot( range(1,min(n_tol,N[0])+1), 1-np.cumsum(d), label='Poisson')

plt.xlim([1, n_tol])
plt.ylim([10**(-6),1])
plt.xscale('log')
plt.yscale('log')
plt.legend(loc="upper right")
if (save_results):
     tikzplotlib.save(model_folder+'outdegree_distribution_fit_ccfd.tikz',  table_row_sep='\\\\\n')
plt.show()

Finally, we report some statistics of the fit with a gamma and a Poisson distribution.

In [ ]:
fit_alpha_alt, fit_loc_alt, fit_beta_alt=stats.gamma.fit(outdegrees, floc=0)
print ('gamma fit data', str(fit_alpha_alt), str(fit_loc_alt), str(fit_beta_alt))
print('gamma', stats.kstest(outdegrees, "gamma", args=(fit_alpha_alt, fit_loc_alt, fit_beta_alt))) 
print('poisson', stats.kstest(outdegrees, "poisson", args=(expected_outdegree[N[0]],0))) 